# Libraries and Configurations

In [5]:
!pip install -r requirements.txt

  Using cached torch-2.8.0-cp313-cp313-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached torchvision-0.23.0-cp313-cp313-manylinux_2_28_x86_64.whl.metadata (6.1 kB)
  Using cached pandas-2.3.1-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (91 kB)
  Using cached opencv_python-4.12.0.88-cp37-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (19 kB)
  Using cached scikit_learn-1.7.1-cp313-cp313-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
  Using cached matplotlib-3.10.5-cp313-cp313-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached PyYAML-6.0.2-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached omegaconf-2.3.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached wandb-0.21.0-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (5

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as L

# Tensor flow addition feature: algo, loss function, 
# import tensorflow_addons as tfa
import glob, random, os, warnings
import matplotlib.pyplot as plt

# Use classification_report for report the f1 score, loss, percisetense, etc 
# Create confuse matrix to regard the modle performance 
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

print('TensorFlow Version ' + tf.__version__)


# Seed the random to consist every train to predictable random value 
def seed_everything(seed = 0):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

seed_everything()
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'pandas'

In [ ]:
image_size = 224
batch_size = 16
n_classes = 5

train_path = './data/processed/train/'
test_path = './data/processed/test'
val_path = './data/processed/val/'



df_train = pd.read_csv('./data/metadata/train_metadata.csv', dtype = 'str')

df_val= pd.read_csv('./data/metadata/val_metadata.csv', dtype = 'str')
df_test = pd.read_csv('./data/metadata/test_metadata.csv', dtype = 'str')


classes = {0 : "COVID",
           1 : "Lung_Opacity",
           2 : "Normal",
           3 : "Viral_Pneumonia",
           }

# Data Augmentations

In [ ]:
def data_augment(image):
    p_spatial = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    p_rotate = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
 
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    
    if p_spatial > .75:
        image = tf.image.transpose(image)
        
    # Rotates
    if p_rotate > .75:
        image = tf.image.rot90(image, k = 3) # rotate 270º
    elif p_rotate > .5:
        image = tf.image.rot90(image, k = 2) # rotate 180º
    elif p_rotate > .25:
        image = tf.image.rot90(image, k = 1) # rotate 90º
        
    return image

# Data Generator

In [ ]:
# Perform gernate image after normalize iamge and data argument 
datagen = tf.keras.preprocessing.image.ImageDataGenerator(samplewise_center = True,
                                                          samplewise_std_normalization = True,
                                                          # Extract data for validation part 
                                                          validation_split = 0.2,
                                                          preprocessing_function = data_augment)
# Extract the training image data as generator 
train_gen = datagen.flow_from_dataframe(dataframe = df_train,
                                        # From data frame training we show the training file path to extract to image 2D value 
                                        directory = ".",
                                        # Image path feature name for image path 
                                        x_col = 'file_path',
                                        # Name label in data frame for image label 
                                        y_col = 'class',
                                        # Get the subset folder 
                                        # subset = 'training',
                                        batch_size = batch_size,
                                        seed = 1,
                                        color_mode = 'rgb',
                                        shuffle = True,
                                        class_mode = 'categorical',
                                        target_size = (image_size, image_size))
# Extract the validation part 
valid_gen = datagen.flow_from_dataframe(dataframe = df_val,
                                        directory = ".",
                                        x_col = 'file_path',
                                        y_col = 'class',
                                        # Extract from validation sub folder 
                                        # subset = 'validation',
                                        batch_size = batch_size,
                                        seed = 1,
                                        color_mode = 'rgb',
                                        shuffle = False,
                                        class_mode = 'categorical',
                                        target_size = (image_size, image_size))

test_gen = datagen.flow_from_dataframe(dataframe = df_test,
                                       x_col = 'file_path',
                                       y_col = None,
                                       batch_size = batch_size,
                                       seed = 1,
                                       color_mode = 'rgb',
                                       shuffle = False,
                                       class_mode = None,
                                       target_size = (image_size, image_size))

### Sample Images Visualization

In [ ]:
# Get 16 image from train data to preview 

images = [train_gen[0][0][i] for i in range(16)]
# Create plot figure 3x5 grid each size 10px square
fig, axes = plt.subplots(3, 5, figsize = (10, 10))

axes = axes.flatten()

for img, ax in zip(images, axes):
    # Make sure the right size 
    ax.imshow(img.reshape(image_size, image_size, 3))
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
a = np.array(train_gen[0])

# Model Hyperparameters

In [ ]:
learning_rate = 0.001
weight_decay = 0.0001
num_epochs = 1

patch_size = 8  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 8
mlp_head_units = [56, 28]  # Size of the dense layers of the final classifier

# Building the Model and it's Components

## 1. Multilayer Perceptron (MLP)

In [ ]:
def mlp(x, hidden_units, dropout_rate):
    # For each hidene unit (list) we create perspective number of dense and dropout 
    for units in hidden_units:
        x = L.Dense(units, activation = tf.nn.gelu)(x)
        x = L.Dropout(dropout_rate)(x)
    # Return the value after go though the layers 
    
    return x

## 2. Patch Creation Layer

In [ ]:
class Patches(L.Layer):

    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        # Number of image as batch size 
        batch_size = tf.shape(images)[0]
        # Reshape to list of patchs 
        patches = tf.image.extract_patches(
            images = images,
            sizes = [1, self.patch_size, self.patch_size, 1],
            # No overlapp each patch 
            strides = [1, self.patch_size, self.patch_size, 1],
            rates = [1, 1, 1, 1],
            padding = 'VALID',
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

### Sample Image Patches Visualization

In [ ]:
plt.figure(figsize=(4, 4))

x = train_gen.next()
image = x[0][0]

plt.imshow(image.astype('uint8'))
plt.axis('off')

resized_image = tf.image.resize(
    tf.convert_to_tensor([image]), size = (image_size, image_size)
)

patches = Patches(patch_size)(resized_image)
print(f'Image size: {image_size} X {image_size}')
print(f'Patch size: {patch_size} X {patch_size}')
print(f'Patches per image: {patches.shape[1]}')
print(f'Elements per patch: {patches.shape[-1]}')

n = int(np.sqrt(patches.shape[1]))
plt.figure(figsize=(4, 4))

for i, patch in enumerate(patches[0]):
    ax = plt.subplot(n, n, i + 1)
    patch_img = tf.reshape(patch, (patch_size, patch_size, 3))
    plt.imshow(patch_img.numpy().astype('uint8'))
    plt.axis('off')

## 3. Patch Encoding Layer

In [ ]:
class PatchEncoder(L.Layer):
    
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        # Project will accept htil
        self.projection = L.Dense(units = projection_dim)
        # Position encoding each image patch 
        self.position_embedding = L.Embedding(
            # Embeding reccept patch and project (down dimension) to vector projection_dimension 
            input_dim = num_patches, output_dim = projection_dim
        )

    def call(self, patch):
        # Concatenation for position encoding 
        positions = tf.range(start = 0, limit = self.num_patches, delta = 1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

## Build the ViT model

In [ ]:
def vision_transformer():
    # Define the input shape 
    inputs = L.Input(shape = (image_size, image_size, 3))
    
    # Create patches. => tranfer input to the patch layer 
    # Contruct define the patch size 
    # Then call by passing the input from previous layer 
    patches = Patches(patch_size)(inputs)
    
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)
    
    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        
        # Layer normalization 1.
        x1 = L.LayerNormalization(epsilon = 1e-6)(encoded_patches)
        
        # Create a multi-head attention layer.
        attention_output = L.MultiHeadAttention(
            num_heads = num_heads, key_dim = projection_dim, dropout = 0.1
        )(x1, x1)
        
        # Skip connection 1.
        x2 = L.Add()([attention_output, encoded_patches])
        
        # Layer normalization 2.
        x3 = L.LayerNormalization(epsilon = 1e-6)(x2)
        
        # MLP.
        x3 = mlp(x3, hidden_units = transformer_units, dropout_rate = 0.1)
        
        # Skip connection 2.
        encoded_patches = L.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = L.LayerNormalization(epsilon = 1e-6)(encoded_patches)
    representation = L.Flatten()(representation)
    representation = L.Dropout(0.5)(representation)
    
    # Add MLP.
    features = mlp(representation, hidden_units = mlp_head_units, dropout_rate = 0.5)
    
    # Classify outputs.
    logits = L.Dense(n_classes)(features)
    
    # Create the model.
    model = tf.keras.Model(inputs = inputs, outputs = logits)
    
    return model

In [ ]:
# Total batch perform calc from number of image and batch size 
decay_steps = train_gen.n // train_gen.batch_size
initial_learning_rate = learning_rate


# Learning rate is the consine funciton base on decay_steps 
lr_decayed_fn = tf.keras.experimental.CosineDecay(initial_learning_rate, decay_steps)
# Make schedule training from this consine decays function for trainning 
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_decayed_fn)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

model = vision_transformer()
    
model.compile(optimizer = optimizer, 
              loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing = 0.1), 
              metrics = ['accuracy'])


STEP_SIZE_TRAIN = train_gen.n // train_gen.batch_size
STEP_SIZE_VALID = valid_gen.n // valid_gen.batch_size

earlystopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy',
                                                 min_delta = 1e-4,
                                                 patience = 5,
                                                 mode = 'max',
                                                 restore_best_weights = True,
                                                 verbose = 1)

checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath = './model.hdf5',
                                                  monitor = 'val_accuracy', 
                                                  verbose = 1, 
                                                  save_best_only = True,
                                                  save_weights_only = True,
                                                  mode = 'max')

callbacks = [earlystopping, lr_scheduler, checkpointer]

model.fit(x = train_gen,
          steps_per_epoch = STEP_SIZE_TRAIN,
          validation_data = valid_gen,
          validation_steps = STEP_SIZE_VALID,
          epochs = num_epochs,
          callbacks = callbacks)

# Model Results

In [ ]:
print('Training results')
model.evaluate(train_gen)

print('Validation results')
model.evaluate(valid_gen)